## Gates

`pyqtorch` implements most of the commonly used gates like Pauli gates, rotation
gates, and controlled gates. Every gate accepts a sequence of `qubits` on which
it operates and a total number `n_qubits` of the state that it will operate on:

In [ ]:
import torch
import pyqtorch.modules as pyq

gate = pyq.X(qubits=[0], n_qubits=1)
z = pyq.zero_state(n_qubits=1)

gate(z)

In [ ]:
gate = pyq.CNOT(qubits=[0,1], n_qubits=2)
z = pyq.zero_state(n_qubits=2)
gate(z)

In [ ]:
z.shape

In `pyqtorch` the state is a `n_qubit+1` dimensional `Tensor`, for example a
state with 3 qubits has the shape `(2, 2, 2, 1)` (i.e. one dimension for each
qubit, plus one dimension for the batch size).


_**NOTE:**_ We always work with batched state in `pyqtorch`.

In [ ]:
z = pyq.zero_state(n_qubits=3)
print(z.shape)
z = pyq.zero_state(n_qubits=3, batch_size=5)
print(z.shape)

## Circuits
### `QuantumCircuit`

To compose multiple gates we use a `QuantumCircuit` which is constructed from
a list of operations.

In [ ]:
circ = pyq.QuantumCircuit(
    n_qubits=2,
    operations=[
        pyq.X([0], 2),
        pyq.CNOT([0,1], 2)
    ]
)

z = pyq.zero_state(2)
circ(z)

Every gate and circuit in `pyqtorch` accepts a state and an optional tensor of angles.
If the gate/circuit does not depend on any angles, the second argument is ignored.

In [ ]:
theta = torch.rand(1)
circ(z, theta)  # theta is ignored

In [ ]:
circ = pyq.QuantumCircuit(
    n_qubits=2,
    operations=[
        pyq.RX([0], 2), # rotation instead of X gate
        pyq.CNOT([0,1], 2)
    ]
)

circ(z, theta)  # theta is used!

The vanilla `QuantumCircuit` is always passing the same `theta` tensor to its operations, meaning
the `forward` method of the circuit is:
```python
class QuantumCircuit(torch.nn.Module):

    # ...

    def forward(self, state: torch.Tensor, thetas: torch.Tensor = None) -> torch.Tensor:
        for op in self.operations:
            state = op(state, thetas)
        return state
```

The `FeaturemapLayer` is a convenience constructor for a `QuantumCircuit` which accepts an operation
to put on every qubit.

In [ ]:
circ = pyq.FeaturemapLayer(n_qubits=3, Op=pyq.RX)
print(circ)

states = pyq.zero_state(n_qubits=3, batch_size=4)
inputs = torch.rand(4)

# the same batch of inputs are passed to the operations
circ(states, inputs).shape

### Trainable `QuantumCircuit`s aka `VariationalLayer`s

If you want the angles of your circuit to be trainable you can use a `VariationalLayer`.
The `VariationalLayer` ignores the second input (because it has trainable angle parameters).

In [ ]:
circ = pyq.VariationalLayer(n_qubits=3, Op=pyq.RX)

state = pyq.zero_state(3)
this_argument_is_ignored = None
circ(state, this_argument_is_ignored)

### Composing `QuantumCircuit`s

As every gate and circuit in `pyqtorch` accept the same arguments we can easily
compose them to larger circuits, i.e. to implement a hardware efficient ansatz:

In [ ]:
def hea(n_qubits: int, n_layers: int) -> pyq.QuantumCircuit:
    ops = []
    for _ in range(n_layers):
        ops.append(pyq.VariationalLayer(n_qubits, pyq.RX))
        ops.append(pyq.VariationalLayer(n_qubits, pyq.RY))
        ops.append(pyq.VariationalLayer(n_qubits, pyq.RX))
        ops.append(pyq.EntanglingLayer(n_qubits))
    return pyq.QuantumCircuit(n_qubits, ops)

circ = hea(3,2)
print(circ)

state = pyq.zero_state(3)
circ(state)